In [1]:
import pygame
import math
import time, copy
from queue import PriorityQueue

pygame.display.set_caption("A* Path Finding")

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
RACK_LOC = [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7),
            (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7),
            (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7),
            (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7),
            (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7),
            (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7)]

TICK = 1
rows = 10
width = 500
win = pygame.display.set_mode((width, width))

RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 255, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE = (255, 165 ,0)
GREY = (128, 128, 128)
TURQUOISE = (64, 224, 208)


In [3]:
# AGV1_PATH = {"start_tick":1, "path":[(2,0), (2,1), (2,1), (1,1), (0,1), (0,1), (0,2), (0,2), (0,2), (0,1), (0,0)]}
# AGV2_PATH = {"start_tick":1, "path":[(1,2), (1,3), (2,1), (1,1), (0,1), (0,1), (0,2), (0,2), (0,2), (0,1), (0,0)]}

In [4]:
class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = rows - col - 1
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows

    def get_pos(self):
        return self.row, self.col

    def is_closed(self):
        return self.color == RED

    def is_open(self):
        return self.color == GREEN

    def is_barrier(self):
        return self.color == BLACK

    def is_start(self):
        return self.color == ORANGE

    def is_end(self):
        return self.color == TURQUOISE

    def reset(self):
        self.color = WHITE

    def make_start(self):
        self.color = ORANGE

    def make_closed(self):
        self.color = RED

    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK

    def make_end(self):
        self.color = TURQUOISE

    def make_path(self):
        self.color = PURPLE

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors = []
        if self.row < self.total_rows - 1 and not grid[self.row + 1][self.col].is_barrier(): # DOWN
            self.neighbors.append(grid[self.row + 1][self.col])

        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier(): # UP
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier(): # RIGHT
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier(): # LEFT
            self.neighbors.append(grid[self.row][self.col - 1])

    def __lt__(self, other):
        return False

In [5]:
def h(p1, p2):
    x1, y1 = p1
    x2, y2 = p2
    return abs(x1 - x2) + abs(y1 - y2)

def reconstruct_path(came_from, current, draw):
    print('reconstruct_path start from:{}'.format(current.get_pos()))
    path = [current.get_pos()]
    while current in came_from:
        current = came_from[current]
        if not isinstance(current, str):            
            current.make_path()
            path.append(current.get_pos())
        else:
            path.append(current)
        draw()  
    path.reverse()    
    return path

def get_turn_point(turn_val):
    turn_pos = turn_val.split('_')
    gap = width // rows    
    return Spot(int(turn_pos[1]), int(turn_pos[2]), gap, rows)
        
def count_reconstruct_path(came_from, current):
    cnt = 0
    while current in came_from:
        current = came_from[current]
        cnt += 1
    return cnt

def check_turn(came_from, current, start):
    ### return False if position didn't move for more than 2
    if count_reconstruct_path(came_from, current) < 3:
        return False
    ### check if current position is already turning (by checking if value of current is string instead of Spot)
    if isinstance(came_from[current], str) or isinstance(came_from[came_from[current]], str):
        print('@@ check_turn already turning!')
        return False
    
    cur_pos = current.get_pos()
    last_pos = came_from[current].get_pos()
    last_last_pos = came_from[came_from[current]].get_pos()
    if last_last_pos[0] == last_pos[0] and cur_pos[0] != last_pos[0] or \
        last_last_pos[1] == last_pos[1] and cur_pos[1] != last_pos[1]:
        return True
    else:
        return False
    

def algorithm(draw, start, end, tick):
    count = 0
    tmp_tick = tick
    open_set = PriorityQueue()
    open_set.put((0, count, start))
    came_from = {}
    g_score = {spot: float("inf") for row in grid for spot in row}
    g_score[start] = 0
    f_score = {spot: float("inf") for row in grid for spot in row}
    f_score[start] = h(start.get_pos(), end.get_pos())
    open_set_hash = {start}
    

    while not open_set.empty():
        time.sleep(0.5)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        prev_open_set = open_set.get()
        current = prev_open_set[2]
        print('@@@@ count_reconstruct_path:{} '.format(count_reconstruct_path(came_from, current)))
        
        tick = count_reconstruct_path(came_from, current) + 1
#         print('@@@ tick: {}  current:{}'.format(tick, current.get_pos()))
        came_from_str = ''
        for k,v in came_from.items():
            if isinstance(k, str):
                came_from_str = '{}, {}:{}'.format(came_from_str, k, v.get_pos())
            elif isinstance(v, str):
                came_from_str = '{}, {}:{}'.format(came_from_str, k.get_pos(), v)
            else:
                came_from_str = '{}, {}:{}'.format(came_from_str, k.get_pos(), v.get_pos())
        print('@@@@ came_from_str:{} '.format(came_from_str))
#         setAGVObstacle(tick, grid, AGV1_PATH)
        open_set_hash.remove(current)

        if current == end:
            path = reconstruct_path(came_from, end, draw)
            return path

        for neighbor in current.neighbors:
            temp_g_score = g_score[current]

            if temp_g_score < g_score[neighbor]:
                if check_turn(came_from, current, start):    ## check if current selected 'current' is a turn
                    ### current: 2,5    neighbor: 1,5     came_from[current]: 3,5
                    ### (3, 5):(3, 4), (2, 5):(3, 5), (1, 5):(2, 5)
                    ### (3, 5):(3, 4), (turn):(3, 5), (2, 5):(turn), (1, 5):(2, 5)
                    print('@@@ turn! current:{} neighbor:{}'.format(current.get_pos(), neighbor.get_pos()))
                    came_from['turn_{}_{}'.format(came_from[current].get_pos()[0], 
                                                  came_from[current].get_pos()[1])] = came_from[current]
                    came_from[current] = 'turn_{}_{}'.format(came_from[current].get_pos()[0], 
                                                             came_from[current].get_pos()[1])
                    came_from[neighbor] = current                      
                else:
                    came_from[neighbor] = current
                g_score[neighbor] = temp_g_score
                f_score[neighbor] = temp_g_score + h(neighbor.get_pos(), end.get_pos())
                if neighbor not in open_set_hash:
                    count += 1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    neighbor.make_open()

        draw()

        if current != start:
            current.make_closed()
            
    return False


def make_grid(rows, width):
    grid = []
    gap = width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Spot(i, rows - j -1, gap, rows)
            grid[i].append(spot)

    return grid


def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GREY, (0, i * gap), (width, i * gap))
        for j in range(rows):
            pygame.draw.line(win, GREY, ((rows - j -1) * gap, 0), ((rows - j -1) * gap, width))


def draw(win, rows, width):
    win.fill(WHITE)

    for row in grid:
        for spot in row:
            spot.draw(win)

    draw_grid(win, rows, width)
    pygame.display.update()


def get_clicked_pos(pos, rows, width):
    gap = width // rows
    y, x = pos

    row = y // gap
    col = x // gap

    return row, col

def add_obstacle(grid, x,y):
    spot = grid[x][y]
    spot.make_barrier()
    spot.update_neighbors(grid)
    for row in grid:
        for s in row:
            s.update_neighbors(grid)
    
def remove_obstacle(grid, x,y):
    spot = grid[x][y]
    spot.reset()
    spot.update_neighbors(grid)
    for row in grid:
        for s in row:
            s.update_neighbors(grid)

def setRack(rack_list):
    for rack_point in rack_list:
        spot = grid[rack_point[0]][rack_point[1]]
        spot.make_barrier()
    for row in grid:
        for s in row:
            s.update_neighbors(grid)
            
def comparePosition(pos1, pos2):
    return (pos1[0] == pos2[0]) and (pos1[1] == pos2[1])
            
def setAGVObstacle(tick, grid, agv):
    start_tick = agv['start_tick']
    agv_path = agv['path']
    tick_interval = tick - start_tick
    if tick_interval >= 0 and tick_interval < len(agv_path):
        add_obstacle(grid, agv_path[tick_interval][0], agv_path[tick_interval][1])
        if tick_interval > 0 and comparePosition(agv_path[tick_interval], agv_path[tick_interval-1]):
            return
        elif tick_interval > 0:
            remove_obstacle(grid, agv_path[tick_interval-1][0], agv_path[tick_interval-1][1])
    

In [6]:

grid = make_grid(rows, width)

if __name__ == "__main__":
    start = None
    end = None

    run = True
    while run:
        draw(win, rows, width)
        for event in pygame.event.get():
            setRack(RACK_LOC)
            if event.type == pygame.QUIT:
                run = False

            if pygame.mouse.get_pressed()[0]: # LEFT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, rows, width)
                spot = grid[row][rows - col - 1]
                if not start and spot != end:
                    start = spot
                    start.make_start()

                elif not end and spot != start:
                    end = spot
                    end.make_end()

#                 elif spot != end and spot != start:
#                     spot.make_barrier()
            

            elif pygame.mouse.get_pressed()[2]: # RIGHT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, rows, width)
                spot = grid[row][col]
                spot.reset()
                if spot == start:
                    start = None
                elif spot == end:
                    end = None

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE and start and end:
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)

                    path = algorithm(lambda: draw(win, rows, width), start, end, TICK)
                    print('@@@@@@@@@@ algorithm result: {}'.format(path))
                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(rows, width)

    pygame.quit()


@@@@ count_reconstruct_path:0 
@@@@ came_from_str: 
@@@@ count_reconstruct_path:1 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0) 
@@@@ count_reconstruct_path:2 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0), (2, 0):(1, 0) 
@@@@ count_reconstruct_path:3 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0), (2, 0):(1, 0), (3, 0):(2, 0) 
@@@@ count_reconstruct_path:4 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0), (2, 0):(1, 0), (3, 0):(2, 0), (4, 0):(3, 0), (3, 1):(3, 0) 
@@@ turn! current:(3, 1) neighbor:(3, 2)
@@@@ count_reconstruct_path:6 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0), (2, 0):(1, 0), (3, 0):(2, 0), (4, 0):(3, 0), (3, 1):turn_3_0, turn_3_0:(3, 0), (3, 2):(3, 1) 
@@ check_turn already turning!
@@@@ count_reconstruct_path:7 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0, 0), (2, 0):(1, 0), (3, 0):(2, 0), (4, 0):(3, 0), (3, 1):turn_3_0, turn_3_0:(3, 0), (3, 2):(3, 1), (3, 3):(3, 2) 
@@@@ count_reconstruct_path:8 
@@@@ came_from_str:, (1, 0):(0, 0), (0, 1):(0

## TODO:
# - tick does not mean 1 move!!